Download links:
- DFDE : https://dfde.efi.int/db/dfde_app.php

# Method :

1. GeoParsing using https://github.com/ArthurCalvi/GeoParsing 
2. Cleaning the name of tree specie

In [1]:
import geopandas as gpd 

dfde = gpd.read_parquet('../data/DFDE/FR/DFDE_1984_2021_EPSG4326_FR.parquet')
dfde

,id,start_date,end_date,cause,area,species,notes,country,location,name,geometry,area_km,reference,reference_long
0,1136,1984-01-01 00:00:00,1984-12-28 00:00:00,Fire,14664.0,None,'Provence-Alpes-Côte d'Azur; Auvergne-Rhône-Al...,France,Republic of France(FR),"intersection: Auvergne-Rhône-Alpes,Midi-Pyréné...","POLYGON ((5.49478 44.11518, 5.46037 44.11834, ...",0.000000,Chevrou R. 1998,"Chevrou, R., 1998. Prévention et lutte contre ..."
1,30369,1984-01-01 00:00:00,1984-12-28 00:00:00,Wind,NaN,None,Mainly oak and beech,France,Vosges(FR),Vosges,"POLYGON ((5.39361 48.39177, 5.39367 48.39171, ...",5897.726024,Guillery 1987,"Guillery, C. (1987). “Introduction.” Revue For..."
2,57306,1984-02-01 00:00:00,1984-02-28 00:00:00,Wind,NaN,None,State forest,France;France;France,Picardie(FR);Bagnoles de l'Orne Normandie(FR);...,Bagnoles-de-l'Orne-Normandie,"POLYGON ((-0.44611 48.56126, -0.44606 48.56117...",15.949458,Aspect Historique 2000,"Aspect Historique de Chablis 2000, ONF"
3,57309,1984-11-01 00:00:00,1984-11-28 00:00:00,Wind,NaN,None,State forest,France;France,Picardie(FR);Bagnoles de l'Orne Normandie(FR),Bagnoles-de-l'Orne-Normandie,"POLYGON ((-0.44611 48.56126, -0.44606 48.56117...",15.949458,Aspect Historique 2000,"Aspect Historique de Chablis 2000, ONF"
4,60061,1984-02-08 00:00:00,1984-02-08 00:00:00,Wind,NaN,None,Haute-Marne,France,Republic of France(FR),Haute-Marne,"POLYGON ((4.62751 48.46717, 4.62751 48.46711, ...",6256.237523,Doll 1988,"Doll D., 1988. Les cataclysms meteorologiques ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,189735,2021-01-01 00:00:00,2021-12-28 00:00:00,Wind,NaN,None,Trend: no change,France,Hesse(FR),Hesse,"POLYGON ((7.01344 48.69227, 7.01369 48.69207, ...",12.961734,AFZ 2022,AFZ Der Wald (2022)
802,189772,2021-01-01 00:00:00,2021-12-28 00:00:00,Frost,NaN,None,late,France,Hesse(FR),Hesse,"POLYGON ((7.01344 48.69227, 7.01369 48.69207, ...",12.961734,AFZ 2022,AFZ Der Wald (2022)
803,189800,2021-01-01 00:00:00,2021-12-28 00:00:00,Beech decline,NaN,Fagus sylvatica,(Buchenrindennekrose);,France,Hesse(FR),Hesse,"POLYGON ((7.01344 48.69227, 7.01369 48.69207, ...",12.961734,AFZ 2022,AFZ Der Wald (2022)
804,189837,2021-01-01 00:00:00,2021-12-28 00:00:00,Viscum album,NaN,Conifers,Trend: increasing,France,Hesse(FR),Hesse,"POLYGON ((7.01344 48.69227, 7.01369 48.69207, ...",12.961734,AFZ 2022,AFZ Der Wald (2022)


## Get Tree-specie

In [26]:
import numpy as np

def get_tree_type(string, dict_tree_type):
    for key in dict_tree_type.keys():
        if string in dict_tree_type[key]:
            return key

    return np.NaN

# Classify each tree species

dict_tree_type_dfde = {
    'mixed': ['Conifers;Broadleaves', 'Broadleaves and conifers (conifers: 51 to 89%)'],
    'conifer': ['Conifers', 'Spruce', 'Maritime pine', 'Pinus maritima', 'Pinus pinaster', 'Fir', 'Pines'],
    'broadleaf': ['Broadleaves', 'Beech', 'Oak', 'Other broadleaves', 'Quercus suber', 'Fagus sylvatica', 'Quercus spp.']
}

dfde['tree_type'] = dfde.species.apply(lambda x: get_tree_type(x, dict_tree_type_dfde)) 
    

## Cleaning essence

In [78]:
# Dictionary to hold translations
translation_dict = {
    'Conifers': 'Conifers',
    'Broadleaves': 'Broadleaves',
    'Beech': 'Beech',
    'Oak': 'Oak',
    'Other broadleaves': 'Other Broadleaves',
    'Spruce': 'Spruce',
    'Maritime pine': 'Maritime Pine',
    'Pinus maritima': 'Maritime Pine',  # Synonym to Maritime pine
    'Quercus suber': 'Cork Oak',
    'Conifers;Broadleaves': 'Conifers;Broadleaves',
    'Pinus pinaster': 'Maritime Pine',
    'Fir': 'Fir',
    'Broadleaves and conifers (conifers: 51 to 89%)': 'Broadleaves and Conifers (conifers: 51 to 89%)',
    'Pines': 'Pines',
    'Fagus sylvatica': 'European Beech',
    'Quercus spp.': 'Oaks'  # Quercus is the genus for all oaks
}

# Print the translation dictionary
dfde['essence'] = dfde.species.apply(lambda x: translation_dict[x].lower() if x in translation_dict.keys() else None)


In [118]:
dfde.to_parquet('../data/processed_datasets/DFDE_1984_2021_EPSG4326_FR.parquet')

# Simplified version

- Adding 'Class' column
- Simplifying geometry at 5km 

In [3]:
import geopandas as gpd
epsg = 'epsg:2154'
dfde = gpd.read_parquet('../data/processed_datasets/DFDE_1984_2021_EPSG4326_FR.parquet').to_crs(epsg)

In [4]:
# DFDE

#class
dict_class = {
    'Fire': ['Fire'],
    'Storm': ['Wind'],
    'Drought': ['Summer drought', 'Frost'],
    'Biotic': [
        'Ips typographus', 'Pissodes spp.', 'Bark beetles', 'Bombix desparate',
        'Zeiraphera diniana', 'Biotic', 'Insects', 'Other insects', 'Biotic;Abiotic',
        'Pityogenes chalcographus', 'Tetropium luridum;Tetropium fuscum',
        'Ips acuminatus', 'Tomicus piniperda;Tomicus minor',
        'Phaenops cyanea', 'Pissodes pini', 'Ips cembrae',
        'Tetropium gabrieli', 'Agrilus biguttatus', 'Agrilus viridis',
        'Xyloterus lineatus', 'Erannis defoliaria',
        'Operophtera brumata;Operophtera fagata', 'Lymantria dispar',
        'Thaumetopoea processionea', 'Hylobius abietis',
        'Melolontha hippocastani;Melolontha melolontha',
        'Microtus agrestis;Microtus arvalis;Clethrionomys glareolus',
        'Arvicola terrestris', 'Lophodermium seditiosum',
        'Sphaeropsis sapinea', 'Heterobasidion annosum',
        'Armillaria mellea', 'Chalara fraxinea;Hymenoscyphus fraxineus',
        'Beech decline', 'Oak decline', 'Viscum album', 'Ips sexdentatus'
    ],
    'Tree-logging': [],
    'Other': ['Accident']
}

def get_class(x):
    for key, values in dict_class.items():
        if x in values:
            return key
    return 'Other'

dfde['class'] = dfde['cause'].apply(get_class)

#geometry
dname_geom = {k:v for k,v in zip(dfde['name'].tolist(), dfde['geometry'].tolist())}
dname_geom = {k:v.buffer(5000).simplify(5000) for k,v in dname_geom.items()}
dname_area = {k:v.area for k,v in dname_geom.items()}

#drop duplicates 
dfde.drop_duplicates(subset=['name', 'start_date', 'end_date', 'essence', 'cause', 'notes'], inplace=True)
dfde['geometry'] = dfde['name'].apply(lambda x: dname_geom[x]) 

#compute area here !
dfde['area'] = dfde['name'].apply(lambda x: dname_area[x])

#clean date
import pandas as pd
dfde['start_date'] = pd.to_datetime(dfde['start_date'])
dfde['end_date'] = pd.to_datetime(dfde['end_date'])

#keep_col
dfde_col = ['start_date', 'end_date', 'geometry', 'class', 'tree_type', 'essence', 'cause', 'notes', 'area']
dfde = dfde[dfde_col]

dfde.dropna(inplace=True)
dfde.drop_duplicates(inplace=True)

groups = dfde.groupby(['start_date', 'end_date', 'cause', 'essence', 'geometry'])

dfde = groups.agg('first').reset_index()
dfde['notes'] = groups['notes'].agg(lambda x: ','.join(set(x))).reset_index()['notes']
#put in a geodataframe
dfde = gpd.GeoDataFrame(dfde, geometry='geometry', crs=epsg)


In [5]:
dfde

,start_date,end_date,cause,essence,geometry,class,tree_type,notes,area
0,1984-01-01,1985-12-28,Ips typographus,spruce,"POLYGON ((899332.033 6504278.695, 916386.336 6...",Biotic,conifer,Vallée de l'Isère; de la Savoie; Haute-Savoie;...,8.340199e+09
1,1984-01-01,1986-12-28,Ips typographus,spruce,"POLYGON ((785984.001 6434917.323, 778975.383 6...",Biotic,conifer,Forêt domaniale du Meygal (Haute-Loire); >2700...,1.132578e+08
2,1984-01-01,1991-12-28,Ips typographus,spruce,"POLYGON ((785984.001 6434917.323, 778975.383 6...",Biotic,conifer,Forêt domaniale du Meygal (Haute-Loire),1.132578e+08
3,1984-07-11,1984-07-11,Wind,beech,"POLYGON ((871607.947 6937228.796, 891400.885 6...",Storm,broadleaf,Briey; Percentage of beech in total wood blown...,7.889669e+09
4,1984-07-11,1984-07-11,Wind,beech,"POLYGON ((815482.473 6818456.330, 827143.074 6...",Storm,broadleaf,Percentage of beech in total wood blown over.,8.455180e+09
...,...,...,...,...,...,...,...,...,...
77,2021-01-01,2021-12-28,Beech decline,european beech,"POLYGON ((991984.368 6854824.724, 1000767.300 ...",Biotic,broadleaf,"(Buchenrindennekrose);,(Buchensterben);Trend: ...",1.028472e+08
78,2021-01-01,2021-12-28,Chalara fraxinea;Hymenoscyphus fraxineus,broadleaves,"POLYGON ((991984.368 6854824.724, 1000767.300 ...",Biotic,broadleaf,Trend: no change,1.028472e+08
79,2021-01-01,2021-12-28,Pissodes pini,pines,"POLYGON ((991984.368 6854824.724, 1000767.300 ...",Biotic,conifer,Trend: increasing,1.028472e+08
80,2021-01-01,2021-12-28,Sphaeropsis sapinea,conifers,"POLYGON ((991984.368 6854824.724, 1000767.300 ...",Biotic,conifer,Trend: increasing,1.028472e+08


In [6]:

dfde.to_parquet('../data/processed_datasets/simplified_DFDE_1984_2021_EPSG2154_FR.parquet')